# Proriol Polynomials 

In this notebook, we construct an explicit form of Proriol polynomialks which can then be copy pasted into a C++code. Look at [PP2017](https://www.jayeshbadwaik.in/doc/public/PP2017.pdf)  for the details on the formulas and calculation.

In [29]:
from mpmath import *

In [54]:
# Degree upto which Proriol Polynomials are to be Generated
N = Integer(3)
# Precision Required (in terms of decimal digits)
dps=10
mp.dps=dps

In [55]:
# Setting the Symbolic Variable for the Variables
eta = var('eta')
xi = var('xi')

In [56]:
def scale_factor(i,j):
    return sqrt(2*(i+1)*(i+j+1))

In [57]:
def mode(k,l):
    return Integer((k+l)*(k+l+1)/2 + l);

In [58]:
def number_of_mode(N):
    return Integer((N+1) *(N+2)/2)

In [59]:
def generate_degree_list(N):
    nmode = number_of_mode(N)
    degree_list  = [None] * nmode
    for i in (0..N):
        for j in (0..i):
            k = j;
            l = i-j;
            mode_index = mode(k,l)
            degree_list[mode_index] = i
    return degree_list

In [60]:
def generate_kllist(N):
    nmode = number_of_mode(N)
    klist = [None] * nmode
    llist = [None] * nmode
    for i in (0..N):
        for j in (0..i):
            k = j;
            l = i-j;
            mode_index = mode(k,l)
            klist[mode_index] = k
            llist[mode_index] = l
    return klist, llist

In [61]:
# Generating the Proriol Polynomials 
nmode = number_of_mode(N)
Pkl = [None] * nmode;
for i in (0..N):
    for j in (0..i):
        k = j;
        l = i-j;
        xiprime = 2 * xi/(1 - eta) - 1
        etaprime = 2*eta - 1
        mode_index = mode(k,l)
        Pkl[mode_index] = legendre_P(k, xiprime)  
        Pkl[mode_index] *= (1-eta)**k 
        Pkl[mode_index] *= jacobi_P(l,2*k+1,0, etaprime)

In [62]:
# Converting the polynomials into symbolic expressions 
R.<xi,eta> = PolynomialRing(QQ,2)
P = [None] * nmode;
coeff_list = [None]*nmode;
degree_list = generate_degree_list(N)
[klist, llist] = generate_kllist(N)

for i in (0..nmode-1):
    P[i] = R(Pkl[i])
    degree = degree_list[i]
    nterm = number_of_mode(degree)
    coeff_list[i] = [None] * nterm
    for j in (0..nterm-1):
        k = klist[j]
        l = llist[j]
        coeff_list[i][j] = P[i].coefficient({xi:k,eta:l})
        
# Generating Derivatives
DP_xi = [None] * nmode;
DP_eta = [None] * nmode;
d_coeff_list = [None]*nmode;

for i in (0..nmode-1):
    DP_xi[i] = P[i].derivative(xi)
    DP_eta[i] = P[i].derivative(eta)
    degree = degree_list[i]
    nterm = number_of_mode(degree)
    d_coeff_list[i] = [None] * nterm
    for j in (0..nterm-1):
        k = klist[j]
        l = llist[j]
        d_coeff_list[i][j] = [DP_xi[i].coefficient({xi:k,eta:l}), DP_eta[i].coefficient({xi:k,eta:l})]

In [63]:
# Output in C++11 Format (x^ky^l) (k,l)  \in {(0,0),(0,1),(1,0),(2,0),(1,1),(0,2)} and so on, in that order.
for i in range(0,nmode):
    print "polynomial_[{}]=".format(i) +"{",
    degree = degree_list[i]
    nterm = number_of_mode(degree)
    for j in range(0,nterm-1):
        print "{}".format(coeff_list[i][j]) + ",",
    j = nterm - 1
    print "{}".format(coeff_list[i][j]), 
    print "};"

polynomial_[0]={ 1 };
polynomial_[1]={ -1, 2, 1 };
polynomial_[2]={ -1, 0, 3 };
polynomial_[3]={ 1, -6, -2, 6, 6, 1 };
polynomial_[4]={ 1, -2, -6, 0, 10, 5 };
polynomial_[5]={ 1, 0, -8, 0, 0, 10 };
polynomial_[6]={ -1, 12, 3, -30, -24, -3, 20, 30, 12, 1 };
polynomial_[7]={ -1, 6, 9, -6, -48, -15, 0, 42, 42, 7 };
polynomial_[8]={ -1, 2, 13, 0, -24, -33, 0, 0, 42, 21 };
polynomial_[9]={ -1, 0, 15, 0, 0, -45, 0, 0, 0, 35 };


In [64]:
# Output in C++11 Format (x^ky^l) (k,l)  \in {(0,0),(0,1),(1,0),(2,0),(1,1),(0,2)} and so on, in that order.
for i in range(0,nmode):
    print "gradient_[{}]=".format(i) +"{",
    degree = degree_list[i]
    nterm = number_of_mode(degree)
    for j in range(0,nterm-1):
        print "{}".format(d_coeff_list[i][j]) + ",",
    j = nterm - 1
    print "{}".format(d_coeff_list[i][j]), 
    print "};"

gradient_[0]={ [0, 0] };
gradient_[1]={ [2, 1], [0, 0], [0, 0] };
gradient_[2]={ [0, 3], [0, 0], [0, 0] };
gradient_[3]={ [-6, -2], [12, 6], [6, 2], [0, 0], [0, 0], [0, 0] };
gradient_[4]={ [-2, -6], [0, 10], [10, 10], [0, 0], [0, 0], [0, 0] };
gradient_[5]={ [0, -8], [0, 0], [0, 20], [0, 0], [0, 0], [0, 0] };
gradient_[6]={ [12, 3], [-60, -24], [-24, -6], [60, 30], [60, 24], [12, 3], [0, 0], [0, 0], [0, 0], [0, 0] };
gradient_[7]={ [6, 9], [-12, -48], [-48, -30], [0, 42], [84, 84], [42, 21], [0, 0], [0, 0], [0, 0], [0, 0] };
gradient_[8]={ [2, 13], [0, -24], [-24, -66], [0, 0], [0, 84], [42, 63], [0, 0], [0, 0], [0, 0], [0, 0] };
gradient_[9]={ [0, 15], [0, 0], [0, -90], [0, 0], [0, 0], [0, 105], [0, 0], [0, 0], [0, 0], [0, 0] };


--------------


This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>